In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[6]').appName('sql-Tranform-demo').config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-360db026-af7e-43a7-a3c0-518885506481;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 255ms :: artifacts dl 9ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------------------------------------

In [14]:
df = spark.read.csv('data/5m-Sales-Records.csv', header = True)
#df = spark.read.csv('s3://feb2025-training-bucket/data/5m-Sales-Records.csv', header = True)

In [15]:
df.show(2)
df.printSchema()
df.count()

+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|  Palau|Office Supplies|       Online|             H|  3/6/2016|517073523|3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25|
|              Europe| Poland|      Beverages|       Online|             L| 4/18/2010|380507028|5/26/2010|      9340|     47.45|    31.79|    443183.00| 296918.60|   146264.40|
+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+---------

5000000

In [8]:
for col_name in df.columns:
    new_col_name = col_name.replace(" ", "_")
    df = df.withColumnRenamed(col_name, new_col_name)
#df.count()
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: string (nullable = true)
 |-- Unit_Price: string (nullable = true)
 |-- Unit_Cost: string (nullable = true)
 |-- Total_Revenue: string (nullable = true)
 |-- Total_Cost: string (nullable = true)
 |-- Total_Profit: string (nullable = true)



In [9]:
# Creating news rows for error records
from pyspark.sql import Row
data = [
    ("Australia and Oceania",None, "Office Supplies", "Online", "H", "3-6-2016", 5555553523, "3/26/2016", -2401, -651.21, 524.96, 15655.21, 1260428.96, 303126.25),
    ("Europe", "Poland", "Beverages", "Online", "L", "12/25/2020", 5555555028, "26/5/2010", -9340, 47.45, -31.79, None, 296918.60, 146264.40)
]

df_new = spark.createDataFrame(data, df.schema)
df_new.show()


+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|   null|Office Supplies|       Online|             H|  3-6-2016|5555553523|3/26/2016|     -2401|   -651.21|   524.96|     15655.21|1260428.96|   303126.25|
|              Europe| Poland|      Beverages|       Online|             L|12/25/2020|5555555028|26/5/2010|     -9340|     47.45|   -31.79|         null|  296918.6|    146264.4|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+---

In [10]:
df_merged = df.union(df_new)
df_merged.createOrReplaceTempView("salesdata")
df_merged.count()

5000002

In [11]:
df_sql = spark.sql("""
    SELECT 
        Region,
        Country,
        Item_Type,
        Sales_Channel,
        Order_Priority,
        Order_Date,
        CAST(Order_ID AS INT) AS Order_ID,
        Ship_Date,
        CAST(Units_Sold AS INT) AS Units_Sold,
        CAST(Unit_Price AS DOUBLE) AS Unit_Price,
        CAST(Unit_Cost AS DOUBLE) AS Unit_Cost,
        CAST(Total_Revenue AS DOUBLE) AS Total_Revenue,
        CAST(Total_Cost AS DOUBLE) AS Total_Cost,
        CAST(Total_Profit AS DOUBLE) AS Total_Profit
    FROM salesdata
""")
df_sql.show()
df_sql.printSchema()


+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|       Palau|Office Supplies|       Online|             H|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25|
|              Europe|      Poland|      Beverages|       Online|             L| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79|     443183.0|  296918.6|    146264.4|
|       North America|      Canada|         Cereal|       Online|             M|  1/8

In [25]:
df_sql = spark.sql("""
    SELECT Region, 
           Country, 
           Total_Revenue, 
           Order_Date, 
           TO_DATE(Order_Date, 'M/d/yyyy') AS Converted_Order_Date,
           DATE_FORMAT(TO_DATE(Order_Date, 'M/d/yyyy'), 'dd-MM-yyyy') AS Formatted_Order_Date
    FROM salesdata
""")
df_sql.createOrReplaceTempView("salesdata_1")
df_sql.count()


5000002

In [18]:
#^[1-9]{1}[0-9]{0,1}: Matches a 1- or 2-digit month (1-12).
#/: Matches the literal separator.
#[1-9]{1}[0-9]{0,1}: Matches a 1- or 2-digit day (1-31).
#[0-9]{4}$: Matches a 4-digit year (e.g., 2015).

df_sql = spark.sql("""
    SELECT * FROM salesdata where Order_Date not RLIKE '^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$';
""")
df_sql.show()
df_sql.count()

+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|   null|Office Supplies|       Online|             H|  3-6-2016|5555553523|3/26/2016|     -2401|   -651.21|   524.96|     15655.21|1260428.96|   303126.25|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+



1

In [19]:
spark.sql(""" 
SELECT * FROM salesdata WHERE `Order_ID` LIKE '555555%'
""").show()

+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|              Europe|Belarus|     Vegetables|       Online|             C| 5/10/2020| 555555284|5/16/2020|         9|    154.06|    90.93|      1386.54|    818.37|      568.17|
|Australia and Oce...|   Fiji|      Household|       Online|             C| 9/25/2012| 555555713|9/27/2012|       136|    668.27|   502.54|     90884.72|  68345.44|    22539.28|
|              Europe|Belarus|     Vegetables|       Online|             C| 5/10/2020| 555555284|5/16/2020|   

In [20]:
#Range Check
spark.sql(""" 
SELECT * FROM salesdata WHERE Units_Sold < 0    OR Total_Revenue < 0   OR Total_Profit < 0 OR Unit_Price < 0;
""").show()



+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|   null|Office Supplies|       Online|             H|  3-6-2016|5555553523|3/26/2016|     -2401|   -651.21|   524.96|     15655.21|1260428.96|   303126.25|
|              Europe| Poland|      Beverages|       Online|             L|12/25/2020|5555555028|26/5/2010|     -9340|     47.45|   -31.79|         null|  296918.6|    146264.4|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+---

In [21]:
#Consistency Check
spark.sql(""" 
SELECT count(*) FROM salesdata WHERE Total_Profit > Total_Revenue
""").show()



+--------+
|count(1)|
+--------+
| 2492996|
+--------+



In [26]:
#Date Format Check
spark.sql(""" 
SELECT * FROM salesdata_1 WHERE CAST(Converted_Order_Date AS DATE) IS NULL; """).show()



+--------------------+-------+-------------+----------+--------------------+--------------------+
|              Region|Country|Total_Revenue|Order_Date|Converted_Order_Date|Formatted_Order_Date|
+--------------------+-------+-------------+----------+--------------------+--------------------+
|Australia and Oce...|   null|     15655.21|  3-6-2016|                null|                null|
+--------------------+-------+-------------+----------+--------------------+--------------------+



In [33]:
error_records_df = spark.sql("""
SELECT *
FROM (
    SELECT *,
        CASE 
            WHEN Order_ID IS NULL THEN 'Order_ID is NULL'
            WHEN Country IS NULL THEN 'Country is NULL'
            WHEN Units_Sold IS NULL THEN 'Units_Sold is NULL'
            WHEN Units_Sold < 0 THEN 'Units_Sold is negative'
            WHEN Total_Revenue < 0 THEN 'Total_Revenue is negative'
            WHEN Total_Profit < 0 THEN 'Total_Profit is negative'
            WHEN Order_Date IS NULL THEN 'Order_Date is NULL'
            WHEN Ship_Date IS NULL THEN 'Ship_Date is NULL'
            WHEN Order_Date NOT RLIKE '^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$' THEN 'Order_Date has invalid format (expected M/d/yyyy)'
            WHEN Ship_Date NOT  RLIKE '^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$' THEN 'Ship_Date has invalid format (expected M/d/yyyy)'
           -- WHEN  Ship_Date < Order_Date THEN 'Ship_Date before Order_Date'
            ELSE NULL
        END AS error_flag
    FROM salesdata
) AS temp
WHERE error_flag IS NOT NULL;
""")
error_records_df.show()
error_records_df.count()

+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+--------------------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|          error_flag|
+--------------------+-------+---------------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+--------------------+
|Australia and Oce...|   null|Office Supplies|       Online|             H|  3-6-2016|5555553523|3/26/2016|     -2401|   -651.21|   524.96|     15655.21|1260428.96|   303126.25|     Country is NULL|
|              Europe| Poland|      Beverages|       Online|             L|12/25/2020|5555555028|26/5/2010|     -9340|     47.45|   -31.79|         null|  296918.6|    146264.4|Units_Sold is neg...|
+----

2

In [31]:
valid_records = spark.sql("""
SELECT *
FROM (
    SELECT *,
        CASE 
            WHEN Order_ID IS NULL THEN 'Order_ID is NULL'
            WHEN Country IS NULL THEN 'Country is NULL'
            WHEN Units_Sold IS NULL THEN 'Units_Sold is NULL'
            WHEN Units_Sold < 0 THEN 'Units_Sold is negative'
            WHEN Total_Revenue < 0 THEN 'Total_Revenue is negative'
            WHEN Total_Profit < 0 THEN 'Total_Profit is negative'
            WHEN Order_Date IS NULL THEN 'Order_Date is NULL'
            WHEN Ship_Date IS NULL THEN 'Ship_Date is NULL'
            WHEN Order_Date NOT RLIKE '^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$' THEN 'Order_Date has invalid format (expected M/d/yyyy)'
            WHEN Ship_Date NOT  RLIKE '^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$' THEN 'Ship_Date has invalid format (expected M/d/yyyy)'
           --WHEN  Ship_Date < Order_Date THEN 'Ship_Date before Order_Date'    
            ELSE NULL
        END AS error_flag
    FROM salesdata
) AS temp
WHERE error_flag IS NULL;
""")
valid_records.createOrReplaceTempView("valid_salesdata")
valid_records.count()


5000000

In [34]:
spark.sql("SELECT *, 'Completed' AS Status FROM salesdata").show()

+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+---------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|   Status|
+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+---------+
|Australia and Oce...|       Palau|Office Supplies|       Online|             H|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25|Completed|
|              Europe|      Poland|      Beverages|       Online|             L| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79|    443183.00| 296918.60|   146264.40|Completed|
|       North America|      Canada|

In [35]:
spark.sql(""" 
SELECT Region, Country, Total_Revenue AS Revenue FROM salesdata;
""").show()


+--------------------+------------+----------+
|              Region|     Country|   Revenue|
+--------------------+------------+----------+
|Australia and Oce...|       Palau|1563555.21|
|              Europe|      Poland| 443183.00|
|       North America|      Canada|  21187.10|
|              Europe|     Belarus| 215748.12|
|Middle East and N...|        Oman|1445453.90|
|  Sub-Saharan Africa|Burkina Faso|1777152.09|
|              Europe|  Montenegro| 109273.01|
|Middle East and N...|  Azerbaijan|3366002.80|
|  Sub-Saharan Africa| South Sudan| 403898.88|
|       North America|   Greenland| 264723.47|
|              Europe|    Portugal|3178444.00|
|  Sub-Saharan Africa|Sierra Leone| 957620.64|
|              Europe|     Germany| 631119.06|
|              Europe|  Montenegro| 505418.32|
|                Asia|  Bangladesh| 661531.20|
|  Sub-Saharan Africa|  Mauritius |1513135.86|
|                Asia|  Kyrgyzstan| 673793.28|
|  Sub-Saharan Africa|  Madagascar| 751300.00|
|            

In [36]:
spark.sql(""" 
SELECT Region, Country, Item_Type, Sales_Channel, Order_Date, Total_Revenue FROM salesdata;
""").show()


+--------------------+------------+---------------+-------------+----------+-------------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Date|Total_Revenue|
+--------------------+------------+---------------+-------------+----------+-------------+
|Australia and Oce...|       Palau|Office Supplies|       Online|  3/6/2016|   1563555.21|
|              Europe|      Poland|      Beverages|       Online| 4/18/2010|    443183.00|
|       North America|      Canada|         Cereal|       Online|  1/8/2015|     21187.10|
|              Europe|     Belarus|         Snacks|       Online| 1/19/2014|    215748.12|
|Middle East and N...|        Oman|         Cereal|      Offline| 4/26/2019|   1445453.90|
|  Sub-Saharan Africa|Burkina Faso|Office Supplies|       Online|  3/3/2012|   1777152.09|
|              Europe|  Montenegro|  Personal Care|       Online|11/24/2012|    109273.01|
|Middle East and N...|  Azerbaijan|      Cosmetics|      Offline| 3/18/2011|   3366002.80|

In [37]:
spark.sql(""" 
SELECT * FROM salesdata WHERE Total_Revenue IS  NULL;
""").show()


+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|Region|Country|Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+
|Europe| Poland|Beverages|       Online|             L|12/25/2020|5555555028|26/5/2010|     -9340|     47.45|   -31.79|         null|  296918.6|    146264.4|
+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+



In [39]:
spark.sql(""" 
SELECT *, COALESCE(Total_Revenue, 0) AS Total_Revenue FROM salesdata WHERE Total_Revenue IS  NULL;
""").show()


+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+-------------+
|Region|Country|Item_Type|Sales_Channel|Order_Priority|Order_Date|  Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|Total_Revenue|
+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+-------------+
|Europe| Poland|Beverages|       Online|             L|12/25/2020|5555555028|26/5/2010|     -9340|     47.45|   -31.79|         null|  296918.6|    146264.4|            0|
+------+-------+---------+-------------+--------------+----------+----------+---------+----------+----------+---------+-------------+----------+------------+-------------+



In [40]:
#Null Check
spark.sql(""" 
SELECT COUNT(*) AS total_rows, 
COUNT(CASE WHEN Country IS NULL THEN 1 END) AS null_country, 
COUNT(CASE WHEN Order_ID IS NULL THEN 1 END) AS null_order_id, 
COUNT(CASE WHEN Total_Revenue IS NULL THEN 1 END) AS null_total_revenue 
FROM salesdata;
""").show()



+----------+------------+-------------+------------------+
|total_rows|null_country|null_order_id|null_total_revenue|
+----------+------------+-------------+------------------+
|   5000002|           1|            0|                 1|
+----------+------------+-------------+------------------+



In [41]:
spark.sql(""" 
SELECT Country, SUM(Total_Revenue) AS Total_Revenue_by_Country 
FROM salesdata 
GROUP BY Country
""").show()


+--------------------+------------------------+
|             Country|Total_Revenue_by_Country|
+--------------------+------------------------+
|              Turkey|    3.607858333332002E10|
|              Angola|    3.593869337192004...|
|          East Timor|       3.606127505553E10|
|             Albania|    3.592686260609997E10|
|           Nicaragua|    3.576133702022005E10|
|               China|    3.601271524842999E10|
|             Somalia|    3.598757324060997E10|
|             Croatia|    3.607999540734997E10|
|             Andorra|    3.597381030885995E10|
|              Norway|    3.583466901126996E10|
|               Niger|    3.598542696423003E10|
|             Ireland|    3.607509419343001...|
|       Liechtenstein|    3.608427412681992E10|
|           Swaziland|    3.580642420097002E10|
|Democratic Republ...|    3.589081698501998E10|
|              Bhutan|    3.609033808057003E10|
|              Israel|    3.601851570352997...|
|            Tunisia |    3.581875832232

In [42]:
spark.sql(""" 
SELECT Country, SUM(Total_Revenue) AS Total_Revenue_by_Country 
FROM salesdata 
GROUP BY Country
ORDER BY Total_Revenue_by_Country DESC;
""").show()


+--------------------+------------------------+
|             Country|Total_Revenue_by_Country|
+--------------------+------------------------+
|              Rwanda|    3.634388914491001E10|
|              Canada|    3.621494398565999E10|
|         Philippines|    3.620237518002995E10|
|    Marshall Islands|    3.615366341424998...|
|              Panama|    3.613639346960005E10|
|             Lesotho|    3.613016405397003E10|
|             Liberia|    3.612976405918994E10|
|              Taiwan|    3.612489163990006E10|
|            Maldives|    3.612321549976002E10|
|Saint Kitts and N...|       3.611518957761E10|
| Trinidad and Tobago|    3.611428475709001E10|
|          Kazakhstan|    3.611396150080001E10|
|            Cameroon|    3.611375133907995...|
|               Japan|    3.611290325129999...|
|         Seychelles |    3.611215496095000...|
|               Tonga|    3.611178305527998E10|
|              Serbia|    3.610944077914002E10|
|              Monaco|    3.610605266115

In [43]:
#Duplicate Check
spark.sql(""" 
SELECT Order_ID, COUNT(*) AS order_count FROM salesdata GROUP BY Order_ID HAVING order_count > 1;
""").show()



25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/18 15:07:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+-----------+
| Order_ID|order_count|
+---------+-----------+
|172520816|          2|
|693221271|          2|
|286153352|          2|
|524290597|          2|
|600975596|          2|
|229758298|          2|
|823808801|          2|
|513141620|          2|
|423505628|          2|
|935651767|          2|
|323637330|          2|
|990212619|          2|
|336575877|          2|
|917588269|          2|
|520938909|          2|
|846018779|          2|
|558573520|          2|
|560909402|          2|
|278614413|          2|
|192224967|          2|
+---------+-----------+
only showing top 20 rows



In [45]:
spark.sql(""" 
SELECT DISTINCT Country,Region,Order_ID FROM salesdata;
""").show()


+--------------------+
|             Country|
+--------------------+
|              Turkey|
|              Angola|
|          East Timor|
|             Albania|
|           Nicaragua|
|               China|
|             Somalia|
|             Croatia|
|             Andorra|
|              Norway|
|               Niger|
|             Ireland|
|       Liechtenstein|
|           Swaziland|
|Democratic Republ...|
|              Bhutan|
|              Israel|
|            Tunisia |
|              Mexico|
|             Armenia|
+--------------------+
only showing top 20 rows



In [46]:
#spark.read.csv('s3://feb2025-training-bucket/data/customers.csv/000.csv', header = True, inferSchema = True).createOrReplaceTempView("customers")
#spark.read.csv('s3://feb2025-training-bucket/data/customerorder_04102024.csv/customerorder.csv', header = True, inferSchema = True).createOrReplaceTempView("customerorder")

spark.read.csv('data/customers.csv', header = True, inferSchema = True).createOrReplaceTempView("customers")
spark.read.csv('data/customerorder_04102024.csv', header = True, inferSchema = True).createOrReplaceTempView("customerorder")

In [47]:

df=spark.sql(""" SELECT * FROM salesdata a JOIN customerorder b ON a.Order_ID = b.Order_ID 
                JOIN customers c ON b.customer_id = c.customer_id """)

df.show(2)

25/02/18 15:10:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+---------+-----------+-----------+------+--------+--------------------+-----+-------+--------+-----------+------+----+-------+--------+------------+----------+--------------------+----------+-------------+---------------+---------------+
|              Region|Country|Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit| Order_ID|customer_id|customer_id|tax_id|tax_code|       customer_name|state|   city|postcode|     street|number|unit| region|district|         lon|       lat|     ship_to_address|valid_from|     valid_to|units_purchased|loyalty_segment|
+--------------------+-------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+---------+----

In [48]:
#convert row to column

spark.sql(""" SELECT Country,Item_Type,
       SUM(CASE WHEN Item_Type = 'Office Supplies' THEN Total_Revenue ELSE 0 END) AS Office_Supplies_Revenue,
       SUM(CASE WHEN Item_Type = 'Beverages' THEN Total_Revenue ELSE 0 END) AS Beverages_Revenue
FROM salesdata
GROUP BY Country,Item_Type """).show()


+--------------------+---------------+-----------------------+--------------------+
|             Country|      Item_Type|Office_Supplies_Revenue|   Beverages_Revenue|
+--------------------+---------------+-----------------------+--------------------+
|           Swaziland|      Beverages|                    0.0|5.3145945450000006E8|
|         South Korea|  Personal Care|                    0.0|                 0.0|
|         Afghanistan|      Cosmetics|                    0.0|                 0.0|
|              Panama|      Household|                    0.0|                 0.0|
|            Barbados|      Baby Food|                    0.0|                 0.0|
|           Macedonia|         Snacks|                    0.0|                 0.0|
|Saint Vincent and...|        Clothes|                    0.0|                 0.0|
|             Germany|         Fruits|                    0.0|                 0.0|
|       Liechtenstein|      Cosmetics|                    0.0|              

In [50]:
result_df = spark.sql("""
SELECT Country, Total_Revenue, Units_Sold,
       RANK() OVER (PARTITION BY Country ORDER BY Total_Revenue DESC, Units_Sold DESC) AS revenue_units_rank
FROM salesdata
""")
result_df.show(5)


+-------+-------------+----------+------------------+
|Country|Total_Revenue|Units_Sold|revenue_units_rank|
+-------+-------------+----------+------------------+
|   null|     15655.21|     -2401|                 1|
|Albania|    999931.76|      3917|                 1|
|Albania|    999931.76|      3917|                 1|
|Albania|    999931.76|      3917|                 1|
|Albania|    999879.30|      2370|                 4|
+-------+-------------+----------+------------------+
only showing top 5 rows

